In [ ]:
# Librerias utiles
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import plotly
import seaborn as sns 
import plotly.express as px
import missingno as msno
import plotly.io as pio
import plotly.graph_objects as go
sns.set_theme()

In [ ]:
victimas = pd.read_csv('reportes/reporte_semujeres_01_OCTUBRE_2021.csv',low_memory=False, dtype={'pk_perfil_agresor': 'object','num_hijos':'int64'}, parse_dates=['fecha_recepcion'])

In [ ]:
victimas.fecha_recepcion

In [ ]:
victimas.fk_euv.value_counts()

## Recortar los primeros 6 digitos de la columna 'Dependencia de recepcion'

In [ ]:
victimas['Dependencia de recepcion'] = victimas['Dependencia de recepcion'].str.slice(start=6)

## Conservar solo los registros en donde la dependencia de recepción haya sido SEMUJERES

In [ ]:
semujeres = victimas[victimas['Dependencia de recepcion'].str.contains("MUJER")==True]

In [ ]:
semujeres.shape

In [ ]:
semujeres.fk_euv.value_counts()

In [ ]:
semujeres[['fk_euv','pk_caso']].value_counts()

## Tenemos 14,011 mujeres y 14,512 casos únicos al dia de la consulta el 23 de agosto 2021
## Tenemos 14,216 mujeres y 14,734 casos únicos al dia de la consulta el 02 de septiembre 2021
## Tenemos 14,489 mujeres y 15,020 casos únicos al dia de la consulta el 01 de octubre 2021

In [ ]:
#semujeres.to_csv('semujeres_dashboard.csv', index=False)

##  DISCAPACIDAD

In [ ]:
discapacidad = pd.read_csv('../BD-sucias/discapacidad.csv')

In [ ]:
discapacidad = discapacidad.iloc[:, [0,1,60,61,62,63,64,65,66,67]]

In [ ]:
discapacidad = discapacidad.rename(columns={'\ncaminar, subir o bajar sus pies?': 'Motriz (piernas)', 'ver (aunque use lentes)?': 'Visual','mover o usar brazos o manos?*':'Motriz (brazos)',
'aprender, recordar o concentrarse?':'Intelectual','escuchar (aunque use aparato auditivo)?':'Auditiva','bañarse, vestirse o comer?':'Psicosocial (independencia)',
'hablar o comunicarse (por ejemplo, entender, o ser entendido por otros)?':'Psicosocial (comunicación)',
'or problemas emocionales o mentales, ¿Cuánta dificultad tiene la víctima para realizar sus actividades diarias (con autonomía e independencia)? Problemas como: autismo, depresión, bipolaridad, esquizofrenia,':'Psicosocial (mentales)'})

In [ ]:
discapacidad.drop_duplicates('fk_euv',keep='last', inplace=True)
discapacidad.reset_index(drop=True, inplace=True)
discapacidad_tipos = discapacidad.iloc[:, 2:]

In [ ]:
#discapacidad.Caminar.unique()

In [ ]:
discapacidad_tipos.replace(to_replace='No indicado ', value='No tiene dificultad', inplace=True)

In [ ]:
discapacidades_tabla = pd.DataFrame()

In [ ]:
for col in discapacidad_tipos.columns:
    new_df = pd.DataFrame(discapacidad_tipos[col].value_counts()).unstack().reset_index()
    discapacidades_tabla=discapacidades_tabla.append(new_df, ignore_index=True)

In [ ]:
discapacidades_tabla = discapacidades_tabla.rename(columns={'level_0':'Tipo de Discapacidad','level_1':'Nivel de dificultad',0:'Cuenta'})

In [ ]:
discapacidades_tabla.replace(to_replace='Motriz (brazos)', value='Motriz', inplace=True)
discapacidades_tabla.replace(to_replace='Motriz (piernas)', value='Motriz', inplace=True)
discapacidades_tabla.replace(to_replace='Psicosocial (comunicación)', value='Psicosocial', inplace=True)
discapacidades_tabla.replace(to_replace='Psicosocial (independencia)', value='Psicosocial', inplace=True)
discapacidades_tabla.replace(to_replace='Psicosocial (mentales)', value='Psicosocial', inplace=True)

In [ ]:
discapacidades_tabla['Porcentaje']=round(discapacidades_tabla.Cuenta/len(discapacidad_tipos)*100,2)

In [ ]:
discapacidades_tabla = discapacidades_tabla[discapacidades_tabla['Nivel de dificultad']!='No tiene dificultad']

In [ ]:
total_mujeres = pd.DataFrame(discapacidades_tabla.groupby('Tipo de Discapacidad').Cuenta.sum()).sort_values('Cuenta', ascending=False)
total_mujeres

In [ ]:
discapacidades_tabla=discapacidades_tabla.groupby(['Tipo de Discapacidad','Nivel de dificultad']).sum().reset_index()
discapacidades_tabla['Tipo de Discapacidad'] = pd.Categorical(discapacidades_tabla['Tipo de Discapacidad'], total_mujeres.index)
discapacidades_tabla= discapacidades_tabla.sort_values('Tipo de Discapacidad').reset_index(drop=True)
discapacidades_tabla

In [ ]:
fig = px.bar(discapacidades_tabla, x="Tipo de Discapacidad", y="Porcentaje", color="Nivel de dificultad", 
title="Gráfica 4. Número de mujeres con discapacidad",
hover_name='Tipo de Discapacidad',
text= "Cuenta",
hover_data={"Cuenta":True, 'Tipo de Discapacidad':False},
labels = {'Cuenta': 'Número de mujeres'},
color_discrete_sequence=px.colors.qualitative.Prism[:5])
fig.update_layout(height=500)
fig.update_traces(textposition='outside', cliponaxis=False)
fig.update_layout(yaxis_ticksuffix = '%')
#print("plotly express hovertemplate:", fig.data[0].hovertemplate)
#fig.update_traces(hovertemplate='<b>%{hovertext}</b><br><br>Tipo de Discapacidad=%{x}<br>Porcentaje=%{y}%<br>Número de mujeres=%{customdata[0]}<extra></extra>')
pio.write_html(fig, file='../docs/figs/g22.html', include_plotlyjs='cdn', full_html=False)
fig.show()

# SERVICIOS

In [ ]:
#servicios = pd.read_csv('../BD-sucias/servicios_23_AGO_2021.csv', low_memory=False)
servicios = pd.read_csv('../BD-sucias/servicios_01_OCTUBRE_2021.csv', low_memory=False)

In [ ]:
servicios.fecha_captura.isna().sum()

In [ ]:
servicios.fecha_captura = pd.to_datetime(servicios.fecha_captura, format='%d/%m/%Y', errors = 'ignore')
servicios.fecha_captura

In [ ]:
servicios.info()

## Conservar aquellos servicios atendidos por SEMUJERES

In [ ]:
servicios.dependenciaquebrindoservicio.unique()

In [ ]:
servicios_semujeres = servicios[servicios['dependenciaquebrindoservicio'].str.contains("MUJER")==True]

In [ ]:
# SERVICIOS
servicios_semujeres = servicios_semujeres[['fk_euv','USUSERVICIO','fk_caso','fecha_captura','serviciodetalle','tiposervicio','estatus','dependenciaquebrindoservicio','numeroservicios']]

In [ ]:
servicios_semujeres.fk_caso = servicios_semujeres.fk_caso.astype(int)
servicios_semujeres.numeroservicios = servicios_semujeres.numeroservicios.astype(int)

In [ ]:
servicios_semujeres.info()

In [ ]:
indicadores_caev = servicios_semujeres.copy()

In [ ]:
indicador_3 = servicios_semujeres.copy()
indicador_3[indicador_3.fk_euv=='3117213094']

In [ ]:
indicador_3[indicador_3.fk_euv=='3117206472']

In [ ]:
len(indicador_3)

In [ ]:
mujeres_concluidas = indicador_3[indicador_3.estatus=='Concluido'].fk_euv.unique()
mujeres_concluidas  = indicador_3[indicador_3.fk_euv.isin(mujeres_concluidas)]
mujeres_concluidas.fk_euv.value_counts()

In [ ]:
mujeres_unicas = mujeres_concluidas.fk_euv.unique()

In [ ]:
mujeres_concluidas

In [ ]:
test = mujeres_concluidas[mujeres_concluidas.fk_euv == '3117206472'].iloc[[0, -1]]
dias = test.fecha_captura.diff().tail(1)
test_df = pd.DataFrame({'fk_euv':'3117206472','dias':dias,'semanas': int(round(dias / np.timedelta64(1, 'W'))),'numeroservicios':test.numeroservicios.tail(1)})
test_df = test_df.reset_index(drop=True)
test_df

In [ ]:
test = mujeres_concluidas[(mujeres_concluidas.fk_euv == '3116900453') & (mujeres_concluidas.fk_caso == 2)]
test

In [ ]:
df = pd.DataFrame(columns=['fk_euv','fk_caso','dias','semanas','numeroservicios'])
for mujer in mujeres_unicas:
    testito = mujeres_concluidas[mujeres_concluidas.fk_euv == mujer]

    casos_concluidos = testito[testito.estatus=='Concluido'].fk_caso.values

    servicios_concluidos = testito[testito.estatus=='Concluido'].serviciodetalle.values

    for caso in casos_concluidos:
        for servicio in servicios_concluidos:
            test = mujeres_concluidas[(mujeres_concluidas.fk_euv == mujer)&(mujeres_concluidas.fk_caso == caso)&(mujeres_concluidas.serviciodetalle == servicio)].iloc[[0, -1]]
            dias = test.fecha_captura.diff().tail(1)
            semanas =  int(round(dias / np.timedelta64(1, 'W')))
            servicios = test.numeroservicios.tail(1)
            test_df = pd.DataFrame({'fk_euv':mujer, 'fk_caso':caso,'dias':dias,'semanas': semanas,'numeroservicios':servicios,'serviciodetalle':servicio})
            df = pd.concat([df, test_df], ignore_index=True)
df

In [ ]:
df['semanas'] = pd.to_numeric(df['semanas'])
df.groupby('serviciodetalle').semanas.mean()

In [ ]:
round(df.semanas.mean())

In [ ]:
fig = px.histogram(df, x = 'semanas',
                title='Gráfica 5. Tiempo promedio transcurrido entre el primer contacto con la usuaria y la conclusión del proceso.',
                #color_discrete_sequence = px.colors.qualitative.Prism,
                #text = 'count',
                #color = 'USUSERVICIO',
                #color = profesionistas.index,
                color_discrete_sequence = [px.colors.qualitative.Prism[1]],
                #color_discrete_sequence= px.colors.sequential.Plasma,
                labels = {'semanas': 'Semanas transcurridas', 'count':'Frecuencia'},
                nbins = 10
                )
fig.update_layout(showlegend = False, yaxis_title="Frecuencia",xaxis_title="Semanas transcurridas", height=500)
#fig.update_layout(title_y='Número de profesionistas')

fig.update_layout(shapes=[
    # adds line at y=5
    dict(
      type= 'line',
      xref= 'x', x0= round(df.semanas.mean()), x1= round(df.semanas.mean()),
      yref= 'paper', y0= 0, y1= 1,
      line=dict(color='white', width=1, dash='dash'),
    )], height=500)

fig.add_annotation(x=round(df.semanas.mean())+1, y=30,
            text="Promedio: {}".format(round(df.semanas.mean())),
            showarrow=False,
            yshift=10,
            font=dict(color='white'))

pio.write_html(fig, file='../docs/figs/g20.html', include_plotlyjs='cdn', full_html=False)


fig.show()

In [ ]:
servicios_semujeres = servicios_semujeres.drop_duplicates(ignore_index=True, subset=['fk_euv','fk_caso','serviciodetalle'], keep='last')
servicios_semujeres = servicios_semujeres.reset_index(drop=True)

In [ ]:
len(servicios_semujeres)

In [ ]:
servicios_semujeres.fk_euv.value_counts()

In [ ]:
servicios_semujeres[servicios_semujeres.fk_euv=='3117213094']

In [ ]:
servicios_semujeres[servicios_semujeres.fk_euv=='3117206472']

In [ ]:
servicios_semujeres.estatus.value_counts()

In [ ]:
round(70/servicios_semujeres.shape[0]*100,2)

In [ ]:
servicios_semujeres.fk_caso.unique()

In [ ]:
servicios_semujeres.serviciodetalle.value_counts()

In [ ]:
servicios_semujeres.shape

In [ ]:
servicios_semujeres.serviciodetalle.unique()

In [ ]:
indicador_servicios = servicios_semujeres.groupby(['serviciodetalle','estatus']).count().numeroservicios.reset_index()
total_servicios = indicador_servicios.groupby('serviciodetalle').sum()
total_servicios

In [ ]:
porcentajes = indicador_servicios.merge(total_servicios, left_on='serviciodetalle', right_on='serviciodetalle')
porcentajes['porcentajes'] = round(100*porcentajes['numeroservicios_x']/porcentajes['numeroservicios_y'],1)
porcentajes

In [ ]:
total_concluidos = porcentajes[porcentajes.estatus=='Concluido'].numeroservicios_x.sum()
porcentajes_total = round(porcentajes[porcentajes.estatus=='Concluido'].numeroservicios_x.sum()/porcentajes[porcentajes.estatus=='Concluido'].numeroservicios_y.sum()*100,2)
print(total_concluidos, porcentajes_total)

In [ ]:
fig = px.bar(porcentajes, x="serviciodetalle", y="porcentajes", color="estatus", 
title="Gráfica 4. Porcentaje de procesos concluidos satisfactoriamente por área de servicio<br><b>Total de servicios concluidos:</b> {}<br><b>Porcentaje de servicios concluidos:</b> {}%".format(total_concluidos, porcentajes_total),
hover_name='serviciodetalle',
text= "numeroservicios_x",
hover_data={"numeroservicios_x":True,'serviciodetalle':False},
labels = {'serviciodetalle': 'Área de servicio', "porcentajes":'Porcentaje', 'estatus':'Estatus','numeroservicios_x':'Número de servicios'},
color_discrete_sequence=['rgb(95, 70, 144)', 'rgb(29, 105, 150)', 'rgb(56, 166, 165)'])
fig.update_traces(textposition='outside',cliponaxis=False)
fig.update_layout(height=500)
fig.update_layout(yaxis_ticksuffix = '%')
#fig.update_layout(yaxis_range=[0, 110])
pio.write_html(fig, file='../docs/figs/g19.html', include_plotlyjs='cdn', full_html=False)
fig.show()

## Tenemos 4956 mujeres únicas y 6244 servicios únicos brindados en la base de datos de servicios al dia de la consulta del 23 de agosto de 2021

## Tenemos 5160 mujeres únicas y 6713 servicios únicos brindados en la base de datos de servicios al dia de la consulta del 02 de septiembre de 2021

In [ ]:
#servicios.to_csv('servicios_dashboard.csv', index=False)

## INDICADORES CEAV

In [ ]:
indicadores_caev.fecha_captura.isna().sum()

In [ ]:
indicadores_caev.info()

In [ ]:
#indicadores_caev['count'] = 1
indicadores_caev['año'] = indicadores_caev['fecha_captura'].dt.year
indicadores_caev['semana'] = indicadores_caev['fecha_captura'].dt.isocalendar().week
usuarios = indicadores_caev.groupby(['USUSERVICIO','semana','año']).numeroservicios

In [ ]:
grup = pd.DataFrame(indicadores_caev.groupby(['USUSERVICIO','fecha_captura']).numeroservicios.count()).reset_index()
grup = grup[grup.USUSERVICIO.str[:4] == 'CYUC']
final = pd.DataFrame()
for user in grup.USUSERVICIO.unique():
    temp = pd.DataFrame(grup[grup.USUSERVICIO==user].groupby([pd.Grouper(freq='W', key='fecha_captura')]).sum()).reset_index()
    temp['USUSERVICIO']=user
    final = final.append(temp, ignore_index=True)

In [ ]:
final.groupby('USUSERVICIO').sum()

In [ ]:
"""ola = usuarios.count()
ola"""

In [ ]:
"""dias = indicadores_caev.groupby(['USUSERVICIO','fecha_captura']).numeroservicios
semana_dif = dias .count().reset_index().sort_values(['USUSERVICIO','fecha_captura'])
semana_dif['semana_dif']=semana_dif.fecha_captura.diff()
semana_dif"""

In [ ]:
"""ola = ola.reset_index()
ola = ola[ola.USUSERVICIO.str[:4] == 'CYUC']
ola = ola.reset_index(drop=True)
ola"""

In [ ]:
#ola[ola.USUSERVICIO=='CYUCIM07'].sum()

In [ ]:
#round(ola[ola.USUSERVICIO=='CYUCIM30'].numeroservicios.mean())

In [ ]:
#ola[ola.USUSERVICIO=='CYUCIM30'].sort_values(['año','semana'])

In [ ]:
"""servicios_semanal = usuarios.count()
servicios_semanal = pd.DataFrame(servicios_semanal)
servicios_semanal = servicios_semanal.reset_index()
servicios_semanal = servicios_semanal[servicios_semanal.USUSERVICIO.str[:4] == 'CYUC']
servicios_semanal = servicios_semanal[ [ 'USUSERVICIO','numeroservicios']]
media_servicios = servicios_semanal.groupby('USUSERVICIO').mean().round().astype(int)
media_servicios = media_servicios.sort_values(['numeroservicios'], ascending=False)
media_servicios"""


In [ ]:
start = final.sort_values(['USUSERVICIO','fecha_captura'])
start = start.drop_duplicates('USUSERVICIO', keep='first').reset_index(drop=True)
end = final.sort_values(['USUSERVICIO','fecha_captura'])
end = final.drop_duplicates('USUSERVICIO', keep='last').reset_index(drop=True)

In [ ]:
# new media servicios
media_servicios = round(final.groupby('USUSERVICIO').mean(),2)
media_servicios

## INSTITUCION (Nombre de usuario)

In [ ]:
nombre_usuario = pd.read_csv('../BD-sucias/institucion_01_OCTUBRE_2021.csv')

In [ ]:
nombre_usuario = nombre_usuario[['fk_usuario', 'dependeciaUsuario','NomUsuario','paterno','materno']]

In [ ]:
nombre_usuario = nombre_usuario.drop_duplicates('fk_usuario',keep='last').reset_index(drop=True)

In [ ]:
nombre_usuario['NomCompleto'] = nombre_usuario['NomUsuario'] + ' ' + nombre_usuario['paterno'] + ' ' + nombre_usuario['materno']

In [ ]:
nombre_usuario = nombre_usuario[['fk_usuario','NomCompleto', 'dependeciaUsuario']]

## MERGE INSTITUCION Y SERVICIOS

In [ ]:
media_servicios = media_servicios.merge(nombre_usuario, left_on=media_servicios.index, right_on='fk_usuario', how='left')

In [ ]:
media_servicios = media_servicios.merge(start[['USUSERVICIO','fecha_captura']], left_on='fk_usuario', right_on='USUSERVICIO', how='left')
media_servicios= media_servicios.drop(columns=['USUSERVICIO'])
media_servicios = media_servicios.rename(columns={'fecha_captura':'Semana de primera captura'})

media_servicios = media_servicios.merge(end[['USUSERVICIO','fecha_captura']], left_on='fk_usuario', right_on='USUSERVICIO', how='left')
media_servicios= media_servicios.drop(columns=['USUSERVICIO'])
media_servicios = media_servicios.rename(columns={'fecha_captura':'Semana de última captura'})

In [ ]:
media_servicios = media_servicios.sort_values('numeroservicios', ascending=False)
media_servicios['dif_capturas'] = media_servicios['Semana de última captura'] - media_servicios['Semana de primera captura']
media_servicios['Semana de última captura'] = media_servicios['Semana de última captura'].dt.date
media_servicios['Semana de primera captura'] = media_servicios['Semana de primera captura'].dt.date
media_servicios['dif_capturas'] = media_servicios['dif_capturas'].dt.days/7 + 1
media_servicios['pesos'] = round(media_servicios.numeroservicios*media_servicios.dif_capturas,2)
media_servicios = media_servicios.sort_values('numeroservicios', ascending=False)
media_servicios


In [ ]:
# nueva media
media_servicios_semanales = round(media_servicios.numeroservicios.mean(),2)
media_servicios_semanales

In [ ]:
media_servicios_semanales_round = round(media_servicios.pesos.mean(),2)
media_servicios_semanales_round

In [ ]:
#profesionistas = profesionistas.reset_index()
fig = px.bar(media_servicios, 
                x = media_servicios.fk_usuario, 
                y = 'numeroservicios', 
                color_continuous_scale='Viridis',
                hover_name='fk_usuario',
                hover_data={'fk_usuario':False, 'NomCompleto':True, 'Semana de primera captura': True, 'Semana de última captura':True, 'numeroservicios': True, 'pesos':True},
                color = 'dif_capturas',
                #color_continuous_midpoint=np.average(media_servicios['numeroservicios'], weights=media_servicios['dif_capturas'].dt.days),
                labels = {'dif_capturas': 'Semanas de antigüedad', "fk_usuario":'Clave del profesionista','NomCompleto':'Nombre'},
                title= 'Gráfica 1. Servicios semanales otorgados por profesionista de la SEMUJERES')

fig.update_layout(showlegend = False, yaxis_title="Servicios Semanales")

fig.update_layout(legend_title= False,height=500)
#fig.update(layout_coloraxis_showscale=False)
fig.update_coloraxes(colorbar_title=None)

#fig.update_traces(textposition='outside', cliponaxis=False)

fig.update_layout(shapes=[
    # adds line at y=5
    dict(
      type= 'line',
      xref= 'paper', x0= 0, x1= 1,
      yref= 'y', y0= media_servicios_semanales, y1= media_servicios_semanales,
      line=dict(color='black', width=1, dash='dash'),
    )])

"""fig.add_annotation(x=77, y=media_servicios_semanales_round,
            text="Promedio de servicios estimados otorgados: {}".format(media_servicios_semanales_round),
            showarrow=False,
            #font_size=14,
            yshift=10)"""

fig.add_annotation(x=77, y=media_servicios_semanales,
            text="Promedio de servicios semanales: {}".format(media_servicios_semanales),
            showarrow=False,
            font_size=14,
            yshift=10)

fig.update_xaxes(
        tickangle = 70)
fig.update_layout(
    coloraxis_colorbar={
        'title':'Semanas<br>Capturando'})
print("plotly express hovertemplate:", fig.data[0].hovertemplate)
fig.update_traces(hovertemplate='<b>%{hovertext}</b><br>Nombre=%{customdata[0]}<br><b>Promedio de Servicios semanales</b>=%{y}<br><b>Servicios estimados</b> (promedio x antigüedad)=%{customdata[3]}<br>Primera captura=%{customdata[1]}<br>Última captura=%{customdata[2]}<br><b>Semanas capturando</b>=%{marker.color} semanas<extra></extra>')
pio.write_html(fig, file='../docs/figs/g15_nueva.html', include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
profesionistas = indicadores_caev.USUSERVICIO.value_counts()
profesionistas = pd.DataFrame(profesionistas)
profesionistas = pd.DataFrame(profesionistas.merge(media_servicios,left_on=profesionistas.index,right_on=media_servicios.fk_usuario,how='left'))
profesionistas = profesionistas.set_index('key_0')
profesionistas = profesionistas[profesionistas.index.str[:4] == 'CYUC']
profesionistas

In [ ]:
print(profesionistas.USUSERVICIO.mean())
print(profesionistas.numeroservicios.mean())

In [ ]:
media_servicios_totales = round(profesionistas.USUSERVICIO.mean())
media_servicios_totales

In [ ]:
#profesionistas = profesionistas.reset_index()
fig = px.bar(profesionistas, 
                x = profesionistas.fk_usuario, 
                y = 'USUSERVICIO', 
                color_continuous_scale='Viridis',
                hover_name='fk_usuario',
                hover_data={'fk_usuario':False, 'NomCompleto':True,'Semana de primera captura': True, 'Semana de última captura':True, 'dif_capturas': True},
                color = 'dif_capturas',
                labels = {'USUSERVICIO': 'Servicios totales', "fk_usuario":'Clave del profesionista', 'NomCompleto': 'Nombre'},
                title= 'Gráfica 2. Servicios totales otorgados por profesionistas de la SEMUJERES')
fig.update_layout(showlegend = False)

#print("plotly express hovertemplate:", fig.data[0].hovertemplate)
fig.update_traces(hovertemplate='<b>%{hovertext}</b><br>Nombre=%{customdata[0]}<br><b>Servicios otorgados</b>=%{y}<br>Primera captura=%{customdata[1]}<br>Última captura=%{customdata[2]}<br>Semanas capturando=%{customdata[3]} semanas<extra></extra>')

fig.update_layout(legend_title= False, height=500)
#fig.update(layout_coloraxis_showscale=False)
fig.update_coloraxes(colorbar_title=None)

fig.update_layout(shapes=[
    # adds line at y=5
    dict(
      type= 'line',
      xref= 'paper', x0= 0, x1= 1,
      yref= 'y', y0= media_servicios_totales, y1= media_servicios_totales,
      line=dict(color='black', width=1, dash='dash'),
    )])

fig.add_annotation(x=77, y=media_servicios_totales,
            text="Promedio de servicios totales: {}".format(media_servicios_totales),
            showarrow=False,
            yshift=10)

fig.update_xaxes(
        tickangle = 70)
        
fig.update_layout(
    coloraxis_colorbar={
        'title':'Semanas<br>Capturando'})

pio.write_html(fig, file='../docs/figs/g16.html', include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
"""fig = px.histogram(profesionistas, x = 'USUSERVICIO',
                title='Gráfica 1. Número total de servicios otorgados por profesionista en SEMUJERES',
                #color_discrete_sequence = px.colors.qualitative.Prism,
                #text = 'count',
                #color = 'USUSERVICIO',
                #color = profesionistas.index,
                color_discrete_sequence = [px.colors.qualitative.Prism[1]],
                #color_discrete_sequence= px.colors.sequential.Plasma,
                labels = {'USUSERVICIO': 'Servicios totales proporcionados'},
                )
fig.update_layout(showlegend = False, yaxis_title="Frecuencia",width=1000, height=500)
#fig.update_layout(title_y='Número de profesionistas')
#pio.write_html(fig, file='figs/g17.html', include_plotlyjs='cdn', full_html=False)
fig.show()"""

In [ ]:
media_servicios = round(profesionistas.numeroservicios.mean())
media_servicios

In [ ]:
profesionistas.shape

In [ ]:
fig = px.histogram(profesionistas, x = 'numeroservicios',
                title='Gráfica 3. Histograma del número de Servicios Semanales otorgados por profesionista en SEMUJERES',
                #color_discrete_sequence = px.colors.qualitative.Prism,
                #text = 'count',
                #color = 'USUSERVICIO',
                #color = profesionistas.index,
                color_discrete_sequence = [px.colors.qualitative.Prism[1]],
                #color_discrete_sequence= px.colors.sequential.Plasma,
                labels = {'numeroservicios': 'Número de Servicios Semanales', 'count':'ola'},
                nbins = 8
                )
fig.update_layout(showlegend = False, yaxis_title="Frecuencia", height=500)
#fig.update_layout(title_y='Número de profesionistas')

fig.update_layout(shapes=[
    # adds line at y=5
    dict(
      type= 'line',
      xref= 'x', x0= media_servicios, x1= media_servicios,
      yref= 'paper', y0= 0, y1= 1,
      line=dict(color='white', width=1, dash='dash'),
    )])

fig.add_annotation(x=media_servicios+1, y=15,
            text="Promedio: {}".format(media_servicios),
            showarrow=False,
            yshift=10,
            font=dict(color='white'))

pio.write_html(fig, file='../docs/figs/g18.html', include_plotlyjs='cdn', full_html=False)


fig.show()

In [ ]:
"""fig = px.histogram(profesionistas, x = 'USUSERVICIO',
                title='Gráfica 1. Histograma de número de servicios proporcionados por profesionista en SEMUJERES',
                color_discrete_sequence = px.colors.qualitative.Prism,
                #text = 'count',
                color = 'USUSERVICIO',
                #color = profesionistas.index,
                #color_discrete_sequence = [px.colors.qualitative.Prism[1]],
                #color_discrete_sequence= px.colors.sequential.Viridis,
                labels = {'USUSERVICIO': 'Núm. de servicios proporcionados', "count":'Frecuencia'},
                )
fig.update_layout(showlegend = False)
fig.show()"""

In [ ]:
"""fig = px.histogram(profesionistas, x = 'numeroservicios',
                title='Gráfica 1. Histograma de número de servicios proporcionados por profesionista en SEMUJERES',
                #color_discrete_sequence = px.colors.qualitative.Prism,
                #text = 'count',
                color = 'numeroservicios',
                #color = profesionistas.index,
                #color_discrete_sequence = [px.colors.qualitative.Prism[1]],
                color_discrete_sequence= px.colors.sequential.Viridis,
                labels = {'numeroservicios': 'Núm. de servicios proporcionados', "count":'Frecuencia'},
                nbins = 5
                )
fig.update_layout(showlegend = False)
fig.show()"""

In [ ]:
#print('Promedio de servicios proporcionados por profesionista',profesionistas.values.mean().round())

# REPLICACION DE GRAFICOS DE LAS FICHAS

In [ ]:
semujeres_2020 = semujeres[semujeres.año_recepcion==2020]

In [ ]:
mun_counts_2020 = pd.DataFrame(semujeres_2020.municipiohechos.value_counts())
mun_counts_2020.reset_index(inplace=True)
mun_counts_2020

In [ ]:
mun_counts = pd.DataFrame(semujeres.municipiohechos.value_counts())
mun_counts.reset_index(inplace=True)
mun_counts

In [ ]:
from mapsmx import MapsMX

In [ ]:
state = MapsMX().get_geo('state')

In [ ]:
muns = MapsMX().get_geo('municipality')

In [ ]:
yuc = muns[muns['cve_ent']=='31']

In [ ]:
import geopandas

In [ ]:
# censo poblacion y vivienda 2020 inegi
inegi_2020 = pd.read_csv('censo_yuc.csv')

In [ ]:
pob_fem = inegi_2020[inegi_2020.NOM_LOC=='Total del Municipio'][['MUN','NOM_MUN','POBFEM' ]].reset_index(drop=True)

In [ ]:
gdf = geopandas.GeoDataFrame(yuc, geometry='geometry_mun')

In [ ]:
gdf_2020 = pd.DataFrame(gdf.merge(mun_counts_2020,left_on='nom_mun',right_on='index',how='left'))
gdf = pd.DataFrame(gdf.merge(mun_counts,left_on='nom_mun',right_on='index',how='left'))

#gdf = pd.DataFrame(gdf.merge(pob_fem,left_on='nom_mun',right_on='NOM_MUN',how='left'))
#gdf_2020

In [ ]:
gdf_2020 = pd.DataFrame(gdf_2020.merge(pob_fem,left_on='nom_mun',right_on='NOM_MUN',how='left'))
#gdf_2020

In [ ]:
#gdf

In [ ]:
gdf = geopandas.GeoDataFrame(gdf, geometry='geometry_mun')
gdf_2020 = geopandas.GeoDataFrame(gdf_2020, geometry='geometry_mun')

In [ ]:
gdf = gdf.set_index('index')
gdf = gdf.rename(columns={'municipiohechos':'Casos registrados'})

gdf_2020 = gdf_2020.set_index('nom_mun')
gdf_2020 = gdf_2020.rename(columns={'municipiohechos':'Casos registrados'})
#gdf

In [ ]:
gdf

In [ ]:
gdf['Casos registrados log'] = np.log10(gdf['Casos registrados'] )
gdf

In [ ]:
gdf_2020['Tasa 1000 casos'] = round(gdf_2020['Casos registrados']/gdf_2020['POBFEM'].astype(int)*1000,2)
gdf_2020

In [ ]:
gdf['Casos registrados'].sum()

In [ ]:
mun_counts.municipiohechos.sum()

In [ ]:
semujeres.municipiohechos.value_counts().sum()

In [ ]:
semujeres.shape

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,10))
#legend = False
gdf.plot(column=gdf['Casos registrados log'], cmap='Blues', linewidth=1, ax=ax, edgecolor='0.9')
plt.ylim([0.88e6, 1.15e6])
plt.xlim([3.7e6, 4e6])
plt.axis('off')
plt.title('Casos Registrados por Municipio')
plt.show()

In [ ]:
print(gdf.crs)

In [ ]:
import pyproj
gdf_crs = gdf.to_crs(pyproj.CRS.from_epsg(4326))
gdf_crs_2020 = gdf_2020.to_crs(pyproj.CRS.from_epsg(4326))

In [ ]:
print(gdf_crs.crs)

In [ ]:
len(semujeres)

In [ ]:
fig = px.choropleth(gdf_crs, geojson=gdf_crs.geometry_mun, 
                    locations=gdf_crs.index, color="Casos registrados log",
                    height=500,
                   color_continuous_scale="Viridis",
                    labels = {'index':'Municipio', "Casos registrados log":'log10'},
                    hover_data={"Casos registrados":True, "Casos registrados log":False},
                    title='Mapa 1. Casos Registrados por Municipio',
                   )
fig.update_geos(fitbounds="locations", visible=False)

fig.update_layout(xaxis=dict(domain=[0, 0.5]), yaxis=dict(domain=[0.25, 0.75]))

fig.add_annotation(
    # The arrow head will be 25% along the x axis, starting from the left
    x=0,
    # The arrow head will be 40% along the y axis, starting from the bottom
    y=0.98,
    text="<b>Temporalidad:</b> {}-{}<br><b>Total de casos:</b> {}".format(min(victimas.año_recepcion),max(victimas.año_recepcion),len(semujeres)),
    showarrow=False,
    bordercolor="black",
    bgcolor="white",
    borderwidth=1.5,
    opacity=0.8
)

fig.update(layout = dict(title=dict(x=0.5)))
fig.update_layout(
    margin={"r":0,"t":30,"l":10,"b":10},
    coloraxis_colorbar={
        'title':'Escala'})
fig.update_layout(title_y=1, title_x=0)
fig.update_layout(coloraxis_showscale=True, height=500)
pio.write_html(fig, file='../docs/figs/mapa.html', include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
gdf_crs_2020['Tasa 1000 casos'] = gdf_crs_2020['Tasa 1000 casos'].fillna(0)
gdf_crs_2020['Casos registrados'] = gdf_crs_2020['Casos registrados'].fillna(0)

In [ ]:
gdf_crs_2020

In [ ]:
"""import json
cols_dd = ["Casos registrados", "POBFEM", "Tasa 1000 casos"]
# we need to add this to select which trace 
# is going to be visible
visible = np.array(cols_dd)
poly_json = json.loads(gdf_crs_2020.geometry_mun.to_json())
# define traces and buttons at once
traces = []
buttons = []
for value in cols_dd:
    traces.append(go.Choropleth(
        geojson=poly_json,
        locations=gdf_crs_2020.index, # Spatial coordinates
        z=gdf_crs_2020[value], # Data to be color-coded
        colorbar_title=value,
        name=value,
        visible= True if value==cols_dd[0] else False))

    buttons.append(dict(label=value,
                        method="update",
                        args=[{"visible":list(visible==value)},
                              {"title":f"<b>{value}</b>"}]))

updatemenus = [{"active":0,
                "buttons":buttons,
               }]


# Show figure
fig = go.Figure(data=traces,
                layout=dict(updatemenus=updatemenus))
# This is in order to get the first title displayed correctly
first_title = cols_dd[0]
fig.update_layout(title=f"<b>{first_title}</b>",title_x=0.5)
fig.update_geos(fitbounds="locations", visible=False)
#lataxis_range=list([0.88e6, 1.15e6]), lonaxis_range=list([3.7e6, 4e6])
pio.write_html(fig, file='../docs/figs/mapa_2_dropdown.html', include_plotlyjs='cdn', full_html=False)
fig.show()"""

In [ ]:
# Hacer menu de seleccion de fecha
fig = px.choropleth(gdf_crs_2020, geojson=gdf_crs_2020.geometry_mun, 
                    locations=gdf_crs_2020.index, color="Tasa 1000 casos",
                    height=500,
                    #color_continuous_scale="Viridis",
                    #color_continuous_midpoint=np.average(gdf_crs_2020['Tasa 1000 casos'], weights=gdf_crs_2020['POBFEM'].astype(int)),
                    labels = {'nom_mun':'Municipio', 'POBFEM':'Población Femenina del Municipio', 'Tasa 1000 casos':'Tasa'},
                    hover_data={"Casos registrados":True,"POBFEM":True,},
                    title='Mapa 2. Tasa de casos registrados por cada 1000 mujeres por Municipio en 2020',
                    )
fig.update_geos(fitbounds="locations", visible=False)

fig.update_layout(xaxis=dict(domain=[0, 0.5]), yaxis=dict(domain=[0.25, 0.75]))

fig.add_annotation(
    # The arrow head will be 25% along the x axis, starting from the left
    x=0,
    # The arrow head will be 40% along the y axis, starting from the bottom
    y=0.98,
    text="<b>Temporalidad:</b> {}<br><b>Total de casos:</b> {}".format(min(semujeres_2020.año_recepcion),len(semujeres_2020)),
    showarrow=False,
    bordercolor="black",
    bgcolor="white",
    borderwidth=1.5,
    opacity=0.8
)

fig.update(layout = dict(title=dict(x=0.5)))
fig.update_layout(
    margin={"r":0,"t":30,"l":10,"b":10},
    coloraxis_colorbar={
        'title':'Tasa'})
fig.update_layout(title_y=1, title_x=0)
fig.update_layout(coloraxis_showscale=True, height=500)
pio.write_html(fig, file='../docs/figs/mapa2.html', include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
#np.average(gdf_crs['Casos registrados'], weights=gdf_crs['POBFEM'].astype(int))

In [ ]:
#np.average(gdf_crs['Casos registrados'])

## Graficos Interactivos

In [ ]:
# ACTIVIDAD/OCUPACION MUJERES



In [ ]:
semujeres.ACTIVIDAD.unique()

In [ ]:
semujeres = pd.concat([semujeres, semujeres.ACTIVIDAD.str.get_dummies(sep=',')], axis=1)

In [ ]:
semujeres.info()

In [ ]:
semujeres.iloc[:,[i for i in range(42,49)]]

In [ ]:
y = [semujeres['Trabaja en el hogar'].sum(), semujeres['Trabaja fuera del hogar'].sum(),semujeres['Estudia'].sum(), semujeres['Jubilada/Pensionada'].sum()+semujeres['Pensionada'].sum(),semujeres['Otro'].sum(),semujeres['Se desconoce'].sum()]
#y_total = sum(y)
y_total = np.array(len(semujeres), dtype=int)
fig = px.bar(x=['Trabaja en el hogar', 'Trabaja fuera del hogar','Estudia','Jubilada/Pensionada','Otro','Se desconoce'], 
            y = y,
            text= np.round(y/y_total*100,2),
            labels = {'x': 'Actividad', "y":'Número de casos', 'text':'Porcentaje'},
            color=px.colors.qualitative.Prism[:6],
            color_discrete_map="identity",
            title='Gráfica 1. Actividad que realizan las mujeres'
            )
fig.update_xaxes(type='category')
fig.update_layout( xaxis={'categoryorder':'total descending'}, xaxis_title=None, height=500)
#fig.update_traces(texttemplate='%{text} %', textposition='outside')
fig.update_traces(texttemplate='%{text} %')
pio.write_html(fig, file='../docs/figs/g1.html', include_plotlyjs='cdn', full_html=False)
#fig.update_xaxes(title='x', visible=False, showticklabels=True)
fig.show()

In [ ]:
px.colors.qualitative.Prism

In [ ]:
semujeres.escolaridad.unique()
validez = semujeres[semujeres.escolaridad=='Estudios que no requieren validéz oficial'].index
tecnica = semujeres[semujeres.escolaridad=='Carrera técnica comercial'].index
semujeres.loc[validez,'escolaridad']='Sin validez'
semujeres.loc[tecnica,'escolaridad']='Carrera técnica'

In [ ]:
semujeres.escolaridad.unique()

In [ ]:
y = semujeres.escolaridad.value_counts().values
y_total = sum(y)
fig = px.bar(x=semujeres.escolaridad.value_counts().index,
            y=semujeres.escolaridad.value_counts().values,
            text= np.round(y/y_total*100,2),
            labels = {'x': 'Escolaridad', "y":'Número de casos', 'text':'Porcentaje'},
            color=px.colors.qualitative.Prism[:11],
            color_discrete_map="identity",
            title='Gráfica 2. Estudios Concluidos'
            )
fig.update_xaxes(type='category')
fig.update_layout(xaxis_title=None, height=500)
fig.update_traces(texttemplate='%{text} %')
#fig.update_xaxes(visible=False, showticklabels=False)
pio.write_html(fig, file='../docs/figs/g2.html',include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
y = semujeres.EstadoCivil.value_counts().values
y_total = sum(y)
fig = px.bar(x=semujeres.EstadoCivil.value_counts().index,
            y=semujeres.EstadoCivil.value_counts().values,
            text= np.round(y/y_total*100,2),
            labels = {'x': 'Estado Civil', "y":'Número de casos', 'text':'Porcentaje'},
            color=px.colors.qualitative.Prism[:8],
            color_discrete_map="identity",
            title='Gráfica 3. Estado Civil'
            )
fig.update_xaxes(type='category')
fig.update_layout(xaxis_title=None, height=500)
fig.update_traces(texttemplate='%{text} %')
#fig.update_xaxes(visible=False, showticklabels=False)
pio.write_html(fig, file='../docs/figs/g3.html',include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
nan = semujeres[semujeres['Habla Indigena'].isna()==True].index
semujeres.loc[nan, 'Habla Indigena'] = 'No'
des = semujeres[semujeres['Habla Indigena']=='Desconocido'].index
semujeres.loc[des, 'Habla Indigena'] = 'No'

In [ ]:
y = semujeres['Habla Indigena'].value_counts().values
y_total = sum(y)
fig = px.bar(x=semujeres['Habla Indigena'].value_counts().index,
            y=semujeres['Habla Indigena'].value_counts().values,
            text= np.round(y/y_total*100,2),
            labels = {'x': 'Habla lengua indígena', "y":'Número de casos', 'text':'Porcentaje'},
            color=px.colors.qualitative.Prism[:2],
            color_discrete_map="identity",
            title='Gráfica 5. Mujeres que hablan Lengua Indígena'
            )
fig.update_xaxes(type='category')
fig.update_layout(xaxis_title=None, height=500)
fig.update_traces(texttemplate='%{text} %')
#fig.update_xaxes(visible=False, showticklabels=False)
pio.write_html(fig, file='../docs/figs/g4.html',include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
y = semujeres['Conocimiento de autoridad'].value_counts().values
y_total = sum(y)
fig = px.bar(x=semujeres['Conocimiento de autoridad'].value_counts().index,
            y=semujeres['Conocimiento de autoridad'].value_counts().values,
            text= np.round(y/y_total*100,2),
            labels = {'x': 'Conocimiento de alguna autoridad', "y":'Número de casos', 'text':'Porcentaje'},
            color=px.colors.qualitative.Prism[:2],
            color_discrete_map="identity",
            title='Gráfica 6. Casos que son conocidos por alguna autoridad'
            )
fig.update_xaxes(type='category')
fig.update_layout( xaxis_title=None, height=500)
fig.update_traces(texttemplate='%{text} %')
#fig.update_xaxes(visible=False, showticklabels=False)
pio.write_html(fig, file='../docs/figs/g5.html',include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
si_conocimiento = semujeres[semujeres['Conocimiento de autoridad']=='Si']

mp = si_conocimiento['Descripcion de autoridad'].str.contains("MINISTERIO").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("MP").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("M.P").sum() 
fg = si_conocimiento['Descripcion de autoridad'].str.contains("FIS").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("FG").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("F.G").sum()
juz = si_conocimiento['Descripcion de autoridad'].str.contains("JUZ").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("JUEZ").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("JZ").sum()
agencia = si_conocimiento['Descripcion de autoridad'].str.contains("AGE").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("AG").sum()
cjm = si_conocimiento['Descripcion de autoridad'].str.contains("JUS").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("CJ").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("JM").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("B1").sum()
prod = si_conocimiento['Descripcion de autoridad'].str.contains("PRODE").sum()
salud = si_conocimiento['Descripcion de autoridad'].str.contains("SALUD").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("HOSP").sum()
judicial = si_conocimiento['Descripcion de autoridad'].str.contains("MUN").sum() + si_conocimiento['Descripcion de autoridad'].str.contains("SSP").sum()
judicial_juzgado = judicial+juz
fis_agencia = fg + agencia

In [ ]:
dic_autoridades = { 'Autoridad':['Ministerio Público','Fiscalía','PRODENNAY', 'CJM','Centro de Salud', 'Poder Judicial'],
                    'Porcentaje':100*np.array([mp, fis_agencia,  prod,  cjm, salud,judicial_juzgado])/sum([mp, fis_agencia,  prod,  cjm, salud, judicial_juzgado]),
                    'Count': np.array([mp, fis_agencia, prod,  cjm, salud, judicial_juzgado])}

In [ ]:
autoridades = pd.DataFrame(dic_autoridades)
autoridades = autoridades.sort_values('Count', ascending=False)
autoridades

In [ ]:
y = autoridades['Count']
y_total = sum(y)
fig = px.bar(x=autoridades['Autoridad'],
            y=autoridades['Count'],
            text= np.round(y/y_total*100,2),
            labels = {'x': 'Autoridad', "y":'Número de casos', 'text':'Porcentaje'},
            color=px.colors.qualitative.Prism[:6],
            color_discrete_map="identity",
            title='Gráfica 7. Autoridad que conoce el caso'
            )
fig.update_xaxes(type='category')
fig.update_layout( xaxis_title=None, height=500)
fig.update_traces(texttemplate='%{text} %')
#fig.update_xaxes(visible=False, showticklabels=False)
pio.write_html(fig, file='../docs/figs/g6.html',include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
semujeres = pd.concat([semujeres, semujeres.tipos.str.get_dummies(sep=',')], axis=1)

In [ ]:
semujeres.descripcion_otro_tipos.unique()

In [ ]:
tipos = semujeres.iloc[:,[i for i in range(49,55)]]
tipos = pd.DataFrame(tipos.sum())
tipos = tipos.sort_values(0,ascending=False)
new_row = pd.Series(data={0:sum(semujeres.descripcion_otro_tipos=='FEMINICIDA')}, name='Feminicida')
#append row to the dataframe
tipos = tipos.append(new_row)

In [ ]:
tipos.loc['Otro',0] = tipos.loc['Otro',0] - tipos.loc['Feminicida',0]

In [ ]:
tipos = tipos.sort_values(0, ascending=False)
tipos

In [ ]:
y = tipos[0].values
y_total = len(semujeres)
fig = px.bar(x=tipos.index,
            y=tipos[0],
            text= np.round(y/y_total*100,2),
            labels = {'x': 'Tipo de violencia', "y":'Número de casos', 'text':'Porcentaje'},
            color=px.colors.qualitative.Prism[:7],
            color_discrete_map="identity",
            title='Gráfica 8. Tipo de violencia registrada'
            )
fig.update_xaxes(type='category')
fig.update_layout( xaxis_title=None,  height=500)
fig.update_traces(texttemplate='%{text} %')
#fig.update_xaxes(visible=False, showticklabels=False)
pio.write_html(fig, file='../docs/figs/g7.html',include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
semujeres.descripcion_otro_tipos

In [ ]:
semujeres.modalidad.unique()

In [ ]:
y = semujeres.modalidad.value_counts().values
y_total = sum(y)
fig = px.bar(x=semujeres.modalidad.value_counts().index,
            y=semujeres.modalidad.value_counts().values,
            text= np.round(y/y_total*100,2),
            labels = {'x': 'Modalidad de la violencia', "y":'Número de casos', 'text':'Porcentaje'},
            color=px.colors.qualitative.Prism[:7],
            color_discrete_map="identity",
            title='Gráfica 9. Modalidad de la violencia'
            )
fig.update_xaxes(type='category')
fig.update_layout( xaxis_title=None,  height=500)
fig.update_traces(texttemplate='%{text} %')
#fig.update_xaxes(visible=False, showticklabels=False)
pio.write_html(fig, file='../docs/figs/g8.html',include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
semujeres['Tipo de vínculo con victima'].unique()

In [ ]:
prismas = px.colors.qualitative.Prism 
prismas += px.colors.qualitative.Prism 

In [ ]:
y = semujeres['Detalle del Tipo Vínculo con victima'].value_counts().values
y_total = sum(y)
fig = px.bar(x=semujeres['Detalle del Tipo Vínculo con victima'].value_counts().index,
            y=semujeres['Detalle del Tipo Vínculo con victima'].value_counts().values,
            text= np.round(y/y_total*100,2),
            labels = {'x': 'Detalle del vínculo con la víctima', "y":'Número de casos', 'text':'Porcentaje'},
            color=prismas[:21],
            color_discrete_map="identity",
            title='Gráfica 11. Detalle del vínculo con la víctima'
            )
fig.update_xaxes(type='category')
fig.update_layout( xaxis={'categoryorder':'total descending'}, height=500)
fig.update_layout( xaxis_title=None)
fig.update_traces(texttemplate='%{text} %')
#fig.update_xaxes(visible=False, showticklabels=False)
pio.write_html(fig, file='../docs/figs/g10.html',include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
fecha='2020-10-01'

In [ ]:
# violencia en los ultimos 12 meses
tuvo_violencia = semujeres[(semujeres['Detalle del Tipo Vínculo con victima'].isin(['Cónyuge o pareja ', 'Ex pareja']))&(semujeres['fecha_recepcion']>=fecha)]
ultimo_año = semujeres[semujeres['fecha_recepcion']>=fecha]
ultimo_año['violencia_ex_pareja'] = 'No'
ultimo_año.loc[tuvo_violencia.index, 'violencia_ex_pareja'] = 'Si'
y = ultimo_año['violencia_ex_pareja'].value_counts().values
y_total = len(ultimo_año)
fig = px.bar(x=ultimo_año['violencia_ex_pareja'].value_counts().index,
            y=ultimo_año['violencia_ex_pareja'].value_counts().values,
            text= np.round(y/y_total*100,2),
            labels = {'x': 'Victima de violencia', "y":'Número de casos', 'text':'Porcentaje'},
            color=px.colors.qualitative.Prism[:2],
            color_discrete_map="identity",
            title="""Gráfica 12. Mujeres que han experimentado violencia por parte de su última pareja en los últimos 12 meses<br><b>Total de casos en los últimos 12 meses:</b> {}""".format(len(ultimo_año))
            )
fig.update_xaxes(type='category')
fig.update_layout( xaxis_title=None, height=500)
fig.update_traces(texttemplate='%{text} %')
"""fig.add_annotation(
    # The arrow head will be 25% along the x axis, starting from the left
    x=0.5,
    # The arrow head will be 40% along the y axis, starting from the bottom
    y=2000,
    text="<b>Total de casos en los últimos 12 meses:</b> {}".format(len(ultimo_año)),
    showarrow=False,
    bordercolor="black",
    bgcolor="white",
    borderwidth=1.5,
    opacity=0.8
)"""
pio.write_html(fig, file='../docs/figs/g21.html',include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
y = semujeres['Tipo de vínculo con victima'].value_counts().values
y_total = sum(y)
fig = px.bar(x=semujeres['Tipo de vínculo con victima'].value_counts().index,
            y=semujeres['Tipo de vínculo con victima'].value_counts().values,
            text= np.round(y/y_total*100,2),
            labels = {'x': 'Tipo de vínculo con la víctima', "y":'Número de casos', 'text':'Porcentaje'},
            color=px.colors.qualitative.Prism[:5],
            color_discrete_map="identity",
            title='Gráfica 10. Tipo de vínculo con la víctima'
            )
fig.update_xaxes(type='category')
fig.update_layout( xaxis_title=None, height=500)
fig.update_traces(texttemplate='%{text} %')
#fig.update_xaxes(visible=False, showticklabels=False)
pio.write_html(fig, file='../docs/figs/g9.html',include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
semujeres['Victima de Trata'].value_counts()

In [ ]:
no = semujeres[semujeres['Victima de Trata']=='Se desconoce'].index
semujeres.loc[no, 'Victima de Trata']= 'No'

In [ ]:
y = semujeres['Victima de Trata'].value_counts().values
y_total = sum(y)
fig = px.bar(x=semujeres['Victima de Trata'].value_counts().index,
            y=semujeres['Victima de Trata'].value_counts().values,
            text= np.round(y/y_total*100,2),
            labels = {'x': 'Victima de Trata', "y":'Número de casos', 'text':'Porcentaje'},
            color=px.colors.qualitative.Prism[:2],
            color_discrete_map="identity",
            title='Gráfica 13. Casos relacionados con el delito de Trata'
            )
fig.update_xaxes(type='category')
fig.update_layout( xaxis_title=None, height=500)
fig.update_traces(texttemplate='%{text} %')
#fig.update_xaxes(visible=False, showticklabels=False)
pio.write_html(fig, file='../docs/figs/g11.html',include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
y = semujeres['Victima de delincuencia'].value_counts().values
y_total = sum(y)
fig = px.bar(x=semujeres['Victima de delincuencia'].value_counts().index,
            y=semujeres['Victima de delincuencia'].value_counts().values,
            text= np.round(y/y_total*100,2),
            labels = {'x': 'Victima de delincuencia', "y":'Número de casos', 'text':'Porcentaje'},
            color=px.colors.qualitative.Prism[:2],
            color_discrete_map="identity",
            title='Gráfica 14. Casos relacionados con Delincuencia Organizada'
            )
fig.update_xaxes(type='category')
fig.update_layout( xaxis_title=None, height=500)
fig.update_traces(texttemplate='%{text} %')
#fig.update_xaxes(visible=False, showticklabels=False)
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [0, 1],
        ticktext = ['No','Si']
    )
)
pio.write_html(fig, file='../docs/figs/g12.html',include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
servicios_semujeres.serviciodetalle.unique()

In [ ]:
y = servicios_semujeres.serviciodetalle.value_counts().values
y_total = sum(y)
fig = px.bar(x=servicios_semujeres.serviciodetalle.value_counts().index,
            y=servicios_semujeres.serviciodetalle.value_counts().values,
            text= np.round(y/y_total*100,2),
            labels = {'x': 'Servicio proporcionado', "y":'Número de casos', 'text':'Porcentaje'},
            color=px.colors.qualitative.Prism[:12],
            color_discrete_map="identity",
            title='Gráfica 15. Servicios proporcionados por la SEMUJERES'
            )
fig.update_xaxes(type='category')
fig.update_layout( xaxis={'categoryorder':'total descending'}, height=500)
fig.update_layout( xaxis_title=None)
fig.update_traces(texttemplate='%{text} %')
pio.write_html(fig, file='../docs/figs/g13.html',include_plotlyjs='cdn', full_html=False)
#fig.update_xaxes(visible=False, showticklabels=False)
fig.show()

In [ ]:
servicios_semujeres.serviciodetalle.value_counts()

In [ ]:
servicios_semujeres.dependenciaquebrindoservicio.unique()

In [ ]:
semujeres.info()

In [ ]:
tipos = semujeres.iloc[:,[49,50,51,52,53,54,28]]
fem = tipos[tipos.descripcion_otro_tipos.isna()==False]
na = tipos[tipos.descripcion_otro_tipos.isna()==True]
tipos.loc[fem.index,'descripcion_otro_tipos']=1
tipos.loc[na.index,'descripcion_otro_tipos']=0
tipos = tipos.rename(columns={'descripcion_otro_tipos':'Feminicida'})

In [ ]:
def matrix(df):
    arr = np.zeros(shape=(7,7), dtype=int)
    
    for index, row in df.iterrows():
        i, j= 0, 0
        #print(row)
        for col_1 in df.columns:
            for col_2 in df.columns:
                if (row[col_1]==1) and (row[col_2] == 1):
                    arr[i][j] += 1
                j += 1
            j = 0
            i += 1
        i = 0
    return arr


In [ ]:
mat = matrix(tipos)
mat

In [ ]:
names = list(tipos.columns)
names

In [ ]:
# chord diagram
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches

import numpy as np

LW = 0.3

def polar2xy(r, theta):
    return np.array([r*np.cos(theta), r*np.sin(theta)])

def hex2rgb(c):
    return tuple(int(c[i:i+2], 16)/256.0 for i in (1, 3 ,5))

def IdeogramArc(start=0, end=60, radius=1.0, width=0.2, ax=None, color=(1,0,0)):
    # start, end should be in [0, 360)
    if start > end:
        start, end = end, start
    start *= np.pi/180.
    end *= np.pi/180.
    # optimal distance to the control points
    # https://stackoverflow.com/questions/1734745/how-to-create-circle-with-b%C3%A9zier-curves
    opt = 4./3. * np.tan((end-start)/ 4.) * radius
    inner = radius*(1-width)
    verts = [
        polar2xy(radius, start),
        polar2xy(radius, start) + polar2xy(opt, start+0.5*np.pi),
        polar2xy(radius, end) + polar2xy(opt, end-0.5*np.pi),
        polar2xy(radius, end),
        polar2xy(inner, end),
        polar2xy(inner, end) + polar2xy(opt*(1-width), end-0.5*np.pi),
        polar2xy(inner, start) + polar2xy(opt*(1-width), start+0.5*np.pi),
        polar2xy(inner, start),
        polar2xy(radius, start),
        ]

    codes = [Path.MOVETO,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.LINETO,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CLOSEPOLY,
             ]

    if ax == None:
        return verts, codes
    else:
        path = Path(verts, codes)
        patch = patches.PathPatch(path, facecolor=color+(0.5,), edgecolor=color+(0.4,), lw=LW)
        ax.add_patch(patch)


def ChordArc(start1=0, end1=60, start2=180, end2=240, radius=1.0, chordwidth=0.7, ax=None, color=(1,0,0)):
    # start, end should be in [0, 360)
    if start1 > end1:
        start1, end1 = end1, start1
    if start2 > end2:
        start2, end2 = end2, start2
    start1 *= np.pi/180.
    end1 *= np.pi/180.
    start2 *= np.pi/180.
    end2 *= np.pi/180.
    opt1 = 4./3. * np.tan((end1-start1)/ 4.) * radius
    opt2 = 4./3. * np.tan((end2-start2)/ 4.) * radius
    rchord = radius * (1-chordwidth)
    verts = [
        polar2xy(radius, start1),
        polar2xy(radius, start1) + polar2xy(opt1, start1+0.5*np.pi),
        polar2xy(radius, end1) + polar2xy(opt1, end1-0.5*np.pi),
        polar2xy(radius, end1),
        polar2xy(rchord, end1),
        polar2xy(rchord, start2),
        polar2xy(radius, start2),
        polar2xy(radius, start2) + polar2xy(opt2, start2+0.5*np.pi),
        polar2xy(radius, end2) + polar2xy(opt2, end2-0.5*np.pi),
        polar2xy(radius, end2),
        polar2xy(rchord, end2),
        polar2xy(rchord, start1),
        polar2xy(radius, start1),
        ]

    codes = [Path.MOVETO,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             ]

    if ax == None:
        return verts, codes
    else:
        path = Path(verts, codes)
        patch = patches.PathPatch(path, facecolor=color+(0.5,), edgecolor=color+(0.4,), lw=LW)
        ax.add_patch(patch)

def selfChordArc(start=0, end=60, radius=1.0, chordwidth=0.7, ax=None, color=(1,0,0)):
    # start, end should be in [0, 360)
    if start > end:
        start, end = end, start
    start *= np.pi/180.
    end *= np.pi/180.
    opt = 4./3. * np.tan((end-start)/ 4.) * radius
    rchord = radius * (1-chordwidth)
    verts = [
        polar2xy(radius, start),
        polar2xy(radius, start) + polar2xy(opt, start+0.5*np.pi),
        polar2xy(radius, end) + polar2xy(opt, end-0.5*np.pi),
        polar2xy(radius, end),
        polar2xy(rchord, end),
        polar2xy(rchord, start),
        polar2xy(radius, start),
        ]

    codes = [Path.MOVETO,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             Path.CURVE4,
             ]

    if ax == None:
        return verts, codes
    else:
        path = Path(verts, codes)
        patch = patches.PathPatch(path, facecolor=color+(0.5,), edgecolor=color+(0.4,), lw=LW)
        ax.add_patch(patch)

def chordDiagram(X, ax, colors=None, width=0.1, pad=2, chordwidth=0.7):
    """Plot a chord diagram
    Parameters
    ----------
    X :
        flux data, X[i, j] is the flux from i to j
    ax :
        matplotlib `axes` to show the plot
    colors : optional
        user defined colors in rgb format. Use function hex2rgb() to convert hex color to rgb color. Default: d3.js category10
    width : optional
        width/thickness of the ideogram arc
    pad : optional
        gap pad between two neighboring ideogram arcs, unit: degree, default: 2 degree
    chordwidth : optional
        position of the control points for the chords, controlling the shape of the chords
    """
    # X[i, j]:  i -> j
    x = X.sum(axis = 1) # sum over rows
    ax.set_xlim(-1.1, 1.1)
    ax.set_ylim(-1.1, 1.1)

    if colors is None:
    # use d3.js category10 https://github.com/d3/d3-3.x-api-reference/blob/master/Ordinal-Scales.md#category10
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
                  '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
        if len(x) > 10:
            print('x is too large! Use x smaller than 10')
        colors = [hex2rgb(colors[i]) for i in range(len(x))]

    # find position for each start and end
    y = x/np.sum(x).astype(float) * (360 - pad*len(x))

    pos = {}
    arc = []
    nodePos = []
    start = 0
    for i in range(len(x)):
        end = start + y[i]
        arc.append((start, end))
        angle = 0.5*(start+end)
        #print(start, end, angle)
        if -30 <= angle <= 210:
            angle -= 90
        else:
            angle -= 270
        nodePos.append(tuple(polar2xy(1.1, 0.5*(start+end)*np.pi/180.)) + (angle,))
        z = (X[i, :]/x[i].astype(float)) * (end - start)
        ids = np.argsort(z)
        z0 = start
        for j in ids:
            pos[(i, j)] = (z0, z0+z[j])
            z0 += z[j]
        start = end + pad

    for i in range(len(x)):
        start, end = arc[i]
        IdeogramArc(start=start, end=end, radius=1.0, ax=ax, color=colors[i], width=width)
        start, end = pos[(i,i)]
        selfChordArc(start, end, radius=1.-width, color=colors[i], chordwidth=chordwidth*0.7, ax=ax)
        for j in range(i):
            color = colors[i]
            if X[i, j] > X[j, i]:
                color = colors[j]
            start1, end1 = pos[(i,j)]
            start2, end2 = pos[(j,i)]
            ChordArc(start1, end1, start2, end2,
                     radius=1.-width, color=colors[i], chordwidth=chordwidth, ax=ax)

    #print(nodePos)
    return nodePos

##################################
if __name__ == "__main__":
    fig = plt.figure(figsize=(5,5))
    flux =  mat

    ax = plt.axes([0,0,1,1])

    nodePos = chordDiagram(flux, ax, colors=[hex2rgb(x) for x in [
 
 '#1d6996', # azul
 '#38a6a5', # azul verdoso
 '#e17c05', # naranja
 '#0f8554', # verde
 '#5f4690', # morado
 '#73af48', # verde claro
 '#edac08', # amarillo

 
 ]])
    #nodePos = chordDiagram(flux, ax)
    ax.axis('off')
    prop = dict(fontsize=16*0.8, ha='center', va='center')
    nodes = names
    for i in range(7):
        ax.text(nodePos[i][0], nodePos[i][1], nodes[i], rotation=nodePos[i][2], **prop)

    plt.title('Gráfica 8.1 Combinaciones de violencias mas comunes', size=15)

    plt.savefig("../docs/figs/chord.png", dpi=600,
            transparent=True,
            bbox_inches='tight', pad_inches=0.02)

In [ ]:
print(px.colors.qualitative.Prism[:6])

In [ ]:
import plotly.graph_objects as go

x = semujeres.EstadoCivil.value_counts().index
y = semujeres.EstadoCivil.value_counts().values

# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=np.round(y/sum(y)*100,2),
            textposition='auto',
        )])
fig.update_traces(texttemplate='%{text} %')
#fig.update_xaxes(visible=False, showticklabels=False)
fig.show()

In [ ]:
y = semujeres.EstadoCivil.value_counts().values
y_total = sum(y)
fig = px.bar(x=semujeres.EstadoCivil.value_counts().index,
            y=semujeres.EstadoCivil.value_counts().values,
            text= np.round(y/y_total*100,1),
            labels = {'x': 'Escolaridad', "y":'Número de casos', 'text':'Porcentaje'},
            color=px.colors.qualitative.Prism[:8],
            color_discrete_map="identity"
            )
fig.update_xaxes(type='category')
#fig.update_layout( xaxis={'categoryorder':'total descending'})
fig.update_traces(texttemplate='%{text} %')
#fig.update_xaxes(visible=False, showticklabels=False)
fig.show()